<a href="https://colab.research.google.com/github/ArunVignesh75/Machine-Learning/blob/main/License_plate_YOLOV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LICENSE PLATE DETECTION & RECOGNITION**

CONTENTS


1.   Importing Packages
2.   Data Pre-processing

1.   Data Augmentation
2.   Spliting the Data


5.   YOLOV8 Model Training
2.   Evaluating the Model


7.   Test Data Predictions
2.   Alphanumeric Data Recognition









In [ ]:
!pip install ultralytics

In [ ]:
!pip install albumentations


In [ ]:
!pip install easyocr

**IMPORTING NESSCESSARY PACKAGES**

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from collections import Counter
from tqdm import tqdm
import shutil
import ultralytics
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from easyocr import Reader
import csv


**DATA PRE-PROCESSING**

In [ ]:
# Path to the training images
image_folder = '/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/Licplatesdetection_train/license_plates_detection_train'

# Function to get image dimensions
def get_image_dimensions(image_path):
    image = cv2.imread(image_path)
    return image.shape[:2]  # Return (height, width)

# Function to resize image to target dimensions
def resize_image(image_path, target_dimensions):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (target_dimensions[1], target_dimensions[0]))  # Note: cv2.resize uses (width, height)
    cv2.imwrite(image_path, resized_image)

# Gather dimensions of all images
dimensions_counter = Counter()
image_paths = []

for filename in os.listdir(image_folder):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_folder, filename)
        dimensions = get_image_dimensions(image_path)
        dimensions_counter[dimensions] += 1
        image_paths.append(image_path)

# Determine the most common dimensions
most_common_dimensions = dimensions_counter.most_common(1)[0][0]

# Resize mismatched images
for image_path in tqdm(image_paths, desc="Resizing images"):
    current_dimensions = get_image_dimensions(image_path)
    if current_dimensions != most_common_dimensions:
        resize_image(image_path, most_common_dimensions)

print(f"All images have been resized to the most common dimensions: {most_common_dimensions}")


In [ ]:

# Paths
image_folder = '/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/Licplatesdetection_train/license_plates_detection_train'
label_folder = '/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/Licplatesdetection_train/license_plates_detection_train_label'

# Ensure output directories exist
os.makedirs(image_folder, exist_ok=True)
os.makedirs(label_folder, exist_ok=True)

# Define the augmentation pipeline
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Function to read YOLO annotation file
def read_yolo_annotation(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()
    bboxes = []
    class_labels = []
    for line in lines:
        parts = line.strip().split()
        class_labels.append(int(parts[0]))
        bboxes.append([float(p) for p in parts[1:]])
    return bboxes, class_labels

# Function to save YOLO annotation file
def save_yolo_annotation(annotation_path, bboxes, class_labels):
    with open(annotation_path, 'w') as file:
        for bbox, class_label in zip(bboxes, class_labels):
            file.write(f'{class_label} ' + ' '.join([f'{p:.6f}' for p in bbox]) + '\n')

# Process each image and its corresponding annotation
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg'):  # Assuming the images are in .jpg format
        image_path = os.path.join(image_folder, filename)
        label_path = os.path.join(label_folder, os.path.splitext(filename)[0] + '.txt')

        # Load image and annotations
        image = cv2.imread(image_path)
        bboxes, class_labels = read_yolo_annotation(label_path)

        # Perform augmentation
        augmented = augmentation_pipeline(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_image = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_class_labels = augmented['class_labels']

        # Save augmented image with a unique filename
        augmented_filename = 'aug_' + filename
        output_image_path = os.path.join(image_folder, augmented_filename)
        cv2.imwrite(output_image_path, aug_image)

        # Save augmented annotation with a unique filename
        output_label_path = os.path.join(label_folder, 'aug_' + os.path.splitext(filename)[0] + '.txt')
        save_yolo_annotation(output_label_path, aug_bboxes, aug_class_labels)

print("Data augmentation completed!")


Data augmentation completed!


**SPLITING THE DATASET**

In [ ]:
train_path_img = '/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/Licplatesdetection_train/license_plates_detection_train'
train_path_label = '/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/Licplatesdetection_train/license_plates_detection_train_label'
val_path_img = '/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/Licplatesdetection_train/license_plates_detection_val_image'
val_path_label = '/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/Licplatesdetection_train/license_plates_detection_val_label'
test_path = '/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/test'

In [ ]:
# Ensure output directories exist
os.makedirs(val_path_img, exist_ok=True)
os.makedirs(val_path_label, exist_ok=True)

# Get list of all image files
image_files = [f for f in os.listdir(train_path_img) if f.endswith('.jpg')]
label_files = [f.replace('.jpg', '.txt') for f in image_files]

# Split into training and validation sets (80:20)
train_images, val_images, train_labels, val_labels = train_test_split(
    image_files, label_files, test_size=0.2, random_state=42
)

# Function to move files
def move_files(files, source_dir, target_dir):
    for file in files:
        shutil.move(os.path.join(source_dir, file), os.path.join(target_dir, file))

# Move validation images and labels
move_files(val_images, train_path_img, val_path_img)
move_files(val_labels, train_path_label, val_path_label)

print("Data split and files moved successfully!")


In [ ]:
ROOT_DIR = '/content/drive/MyDrive/Yolov/LICENSE_DATA'

**TRAINING THE MODEL**

In [ ]:
model = YOLO('yolov8s.pt')
results = model.train(data=os.path.join(ROOT_DIR, "data.yaml"), epochs=10, imgsz=640)

100%|██████████| 21.5M/21.5M [00:00<00:00, 245MB/s]


Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Yolov/LICENSE_DATA/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

100%|██████████| 755k/755k [00:00<00:00, 22.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 95.5MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


...

      10/10      4.14G     0.9453     0.5205     0.9833         15        640: 100%|██████████| 90/90 [00:43<00:00,  2.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:04<00:00,  2.55it/s]

                   all        360        360      0.984      0.989      0.991      0.752



10 epochs completed in 0.228 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:08<00:00,  1.45it/s]


                   all        360        360      0.984      0.989      0.991      0.752
Speed: 0.5ms preprocess, 3.8ms inference, 0.0ms loss, 4.5ms postprocess per image
Results saved to runs/detect/train


**USING TRAINED MODEL**

In [ ]:
# Assuming 'best.pt' is saved at 'runs/train/exp1/weights/best.pt'
trained_model = YOLO('/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/runs/detect/train2/weights/best.pt')


**VALIDATING THE MODEL**

In [ ]:
# Validate the model
metrics = trained_model.val(data=os.path.join(ROOT_DIR, "data.yaml"), split="val", visualize=True)
# no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 14.6MB/s]
val: Scanning /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/labels/val.cache... 360 images, 0 backgrounds, 0 corrupt: 100%|██████████| 360/360 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [04:03<00:00, 10.60s/it]


                   all        360        360      0.904      0.867      0.929      0.585
Speed: 1.6ms preprocess, 634.2ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val


array([    0.58471])

In [ ]:
# Assuming 'test' section in data.yaml defines test data path
test_data_path = "/content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test"

**PREDICTING THE TEST SET**

In [ ]:
# Make predictions on test data and save results
results = trained_model.predict(test_data_path, save_crop = True)



image 1/210 /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/1000.jpg: 448x640 3 license_plates, 739.3ms
image 2/210 /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/1001.jpg: 640x384 1 license_plate, 411.0ms
image 3/210 /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/1002.jpg: 640x480 2 license_plates, 973.4ms
image 4/210 /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/1003.jpg: 384x640 1 license_plate, 956.5ms
image 5/210 /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/1004.jpg: 640x384 1 license_plate, 1554.9ms
image 6/210 /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/1005.jpg: 640x384 1 license_plate, 1395.2ms
image 7/210 /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/1006.jpg: 640x480 1 license_plate, 2209.2ms
image 8/210 /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/1007.jpg: 640x384 1 license_plate, 1067.4ms
image 9/210 /content/drive/MyDrive/Yolov/LICENSE_DATA/Data/test/test/1008.jpg: 384x640 1 license_

**LICENSE PLATE RECOGNITION**

In [ ]:
# Define path to folder containing cropped license plates
plate_dir = "/content/runs/detect/predict/crops/license_plate"

# Define output folder to save recognized text
output_dir = "/content/drive/MyDrive/Yolov/LICENSE_DATA/plate_recognition"

# Initialize EasyOCR reader (adjust language code if needed)
reader = Reader(['ar', 'en'])  # Arabic and English languages

# Loop through each image in the directory
for filename in os.listdir(plate_dir):
  if filename.endswith(".jpg") or filename.endswith(".png"):
    # Construct image path
    img_path = os.path.join(plate_dir, filename)

    # Perform text recognition
    result = reader.readtext(img_path, detail=0)

    # Extract recognized text (handle empty results)
    if result:
      recognized_text = " ".join(result)
    else:
      recognized_text = "N/A"  # Set default text if no recognition

    # Construct output file path (remove extension and add .txt)
    output_filename = os.path.splitext(filename)[0] + ".txt"  # Get filename without extension
    output_path = os.path.join(output_dir, output_filename)

    # Save recognized text to file
    with open(output_path, "w") as f:
      f.write(recognized_text)

    print(f"Text recognized for {filename} and saved to {output_filename}")

print("Text recognition completed for all images.")


Text recognized for 1016.jpg and saved to 1016.txt
Text recognized for 1047.jpg and saved to 1047.txt
Text recognized for 1091.jpg and saved to 1091.txt
Text recognized for 1031.jpg and saved to 1031.txt
Text recognized for 10683.jpg and saved to 10683.txt
Text recognized for 1085.jpg and saved to 1085.txt
Text recognized for 1094.jpg and saved to 1094.txt
Text recognized for 1071.jpg and saved to 1071.txt
Text recognized for 919.jpg and saved to 919.txt
Text recognized for 9212.jpg and saved to 9212.txt
Text recognized for 1049.jpg and saved to 1049.txt
Text recognized for 10942.jpg and saved to 10942.txt
Text recognized for 1074.jpg and saved to 1074.txt
Text recognized for 931.jpg and saved to 931.txt
Text recognized for 10332.jpg and saved to 10332.txt
Text recognized for 930.jpg and saved to 930.txt
Text recognized for 934.jpg and saved to 934.txt
Text recognized for 9302.jpg and saved to 9302.txt
Text recognized for 10682.jpg and saved to 10682.txt
Text recognized for 1040.jpg an